In [1]:
from __future__ import print_function

import tensorflow as tf
from tensorflow.python.framework import graph_util
from tensorflow.python.platform import gfile
from tensorflow.core.framework import graph_pb2

In [2]:
def display_nodes(nodes):
    for i, node in enumerate(nodes):
        print('%d %s %s' % (i, node.name, node.op))
        [print(u'└─── %d ─ %s' % (i, n)) for i, n in enumerate(node.input)]

In [56]:
graph = tf.GraphDef()
with tf.gfile.Open('/Users/Yoann/Downloads/output_graph_fruits_hidden_1.0_224_q.pb', 'r') as f:
    data = f.read()
    graph.ParseFromString(data)
    
display_nodes(graph.node)

0 MobilenetV1/Predictions/Reshape/shape Const
1 MobilenetV1/Conv2d_0/weights/read/_82__cf__82 Const
2 input Placeholder
3 MobilenetV1/MobilenetV1/Conv2d_0/BatchNorm/batchnorm/mul_1 Conv2D
└─── 0 ─ input
└─── 1 ─ MobilenetV1/Conv2d_0/weights/read/_82__cf__82
4 MobilenetV1/MobilenetV1/Conv2d_0/BatchNorm/batchnorm/add_1 Add
└─── 0 ─ MobilenetV1/MobilenetV1/Conv2d_0/BatchNorm/batchnorm/mul_1
└─── 1 ─ MobilenetV1/MobilenetV1/Conv2d_0/BatchNorm/batchnorm/sub/_81__cf__81
5 MobilenetV1/MobilenetV1/Conv2d_0/Relu6 Relu6
└─── 0 ─ MobilenetV1/MobilenetV1/Conv2d_0/BatchNorm/batchnorm/add_1
6 MobilenetV1/MobilenetV1/Conv2d_1_depthwise/depthwise DepthwiseConv2dNative
└─── 0 ─ MobilenetV1/MobilenetV1/Conv2d_0/Relu6
└─── 1 ─ MobilenetV1/Conv2d_1_depthwise/depthwise_weights/read/_79__cf__79
7 MobilenetV1/MobilenetV1/Conv2d_1_depthwise/BatchNorm/batchnorm/mul_1 Mul
└─── 0 ─ MobilenetV1/MobilenetV1/Conv2d_1_depthwise/depthwise
└─── 1 ─ MobilenetV1/MobilenetV1/Conv2d_1_depthwise/BatchNorm/batchnorm/mul/_77

In [17]:
mnist.test.images[:256]

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32)

In [57]:
nodes = graph.node
del nodes[220]
del nodes[219]
del nodes[218]
del nodes[217]
del nodes[216]
del nodes[215]
del nodes[214]
del nodes[213]
del nodes[212]
del nodes[211]
del nodes[210]
del nodes[201]

In [62]:
nodes[213].input[0] = "final_training_ops/hidden/Wx_plus_b/Relu"

In [63]:
display_nodes(nodes)

0 MobilenetV1/Predictions/Reshape/shape Const
1 MobilenetV1/Conv2d_0/weights/read/_82__cf__82 Const
2 input Placeholder
3 MobilenetV1/MobilenetV1/Conv2d_0/BatchNorm/batchnorm/mul_1 Conv2D
└─── 0 ─ input
└─── 1 ─ MobilenetV1/Conv2d_0/weights/read/_82__cf__82
4 MobilenetV1/MobilenetV1/Conv2d_0/BatchNorm/batchnorm/add_1 Add
└─── 0 ─ MobilenetV1/MobilenetV1/Conv2d_0/BatchNorm/batchnorm/mul_1
└─── 1 ─ MobilenetV1/MobilenetV1/Conv2d_0/BatchNorm/batchnorm/sub/_81__cf__81
5 MobilenetV1/MobilenetV1/Conv2d_0/Relu6 Relu6
└─── 0 ─ MobilenetV1/MobilenetV1/Conv2d_0/BatchNorm/batchnorm/add_1
6 MobilenetV1/MobilenetV1/Conv2d_1_depthwise/depthwise DepthwiseConv2dNative
└─── 0 ─ MobilenetV1/MobilenetV1/Conv2d_0/Relu6
└─── 1 ─ MobilenetV1/Conv2d_1_depthwise/depthwise_weights/read/_79__cf__79
7 MobilenetV1/MobilenetV1/Conv2d_1_depthwise/BatchNorm/batchnorm/mul_1 Mul
└─── 0 ─ MobilenetV1/MobilenetV1/Conv2d_1_depthwise/depthwise
└─── 1 ─ MobilenetV1/MobilenetV1/Conv2d_1_depthwise/BatchNorm/batchnorm/mul/_77

In [64]:
output_graph = graph_pb2.GraphDef()
output_graph.node.extend(nodes)
with tf.gfile.GFile('../model/fruits/output_graph_fruits_hidden_1.0_224_q_without_dropout.pb', 'w') as f:
    f.write(output_graph.SerializeToString())

In [65]:
import cv
import numpy as np
x = cv.LoadImageM('banana.jpeg')
im = np.asarray(x)[1:, 1:, :].reshape(1, 224, 224, 3)
im.shape

(1, 224, 224, 3)

In [68]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0])

In [67]:
def test_graph(graph_path, use_dropout):
    tf.reset_default_graph()
    graph_def = tf.GraphDef()
    
    with tf.gfile.FastGFile(graph_path, 'rb') as f:
        graph_def.ParseFromString(f.read())
        
    _ = tf.import_graph_def(graph_def, name='')
    sess = tf.Session()    
    prediction_tensor = sess.graph.get_tensor_by_name('final_result_fruits:0') 
    
    feed_dict = {'input:0': im}
    if use_dropout:
        feed_dict['keep_prob:0'] = 1.0
        
    predictions = sess.run(prediction_tensor, feed_dict)
    result = accuracy(predictions, mnist.test.labels[:256])
    return result

In [69]:
result_1 = test_graph('/Users/Yoann/Downloads/output_graph_fruits_hidden_1.0_224_q.pb', use_dropout=True)

In [70]:
result_1

1900.0

In [71]:
result_2 = test_graph('../model/fruits/output_graph_fruits_hidden_1.0_224_q_without_dropout.pb', use_dropout=False)

In [72]:
result_2

1900.0

In [4]:
graph = tf.GraphDef()
with tf.gfile.Open('../model/fruits/output_graph_fruits_hidden_1.0_224_q_without_dropout.pb', 'r') as f:
    data = f.read()
    graph.ParseFromString(data)
    
display_nodes(graph.node)

0 MobilenetV1/Predictions/Reshape/shape Const
1 MobilenetV1/Conv2d_0/weights/read/_82__cf__82 Const
2 input Placeholder
3 MobilenetV1/MobilenetV1/Conv2d_0/BatchNorm/batchnorm/mul_1 Conv2D
└─── 0 ─ input
└─── 1 ─ MobilenetV1/Conv2d_0/weights/read/_82__cf__82
4 MobilenetV1/MobilenetV1/Conv2d_0/BatchNorm/batchnorm/add_1 Add
└─── 0 ─ MobilenetV1/MobilenetV1/Conv2d_0/BatchNorm/batchnorm/mul_1
└─── 1 ─ MobilenetV1/MobilenetV1/Conv2d_0/BatchNorm/batchnorm/sub/_81__cf__81
5 MobilenetV1/MobilenetV1/Conv2d_0/Relu6 Relu6
└─── 0 ─ MobilenetV1/MobilenetV1/Conv2d_0/BatchNorm/batchnorm/add_1
6 MobilenetV1/MobilenetV1/Conv2d_1_depthwise/depthwise DepthwiseConv2dNative
└─── 0 ─ MobilenetV1/MobilenetV1/Conv2d_0/Relu6
└─── 1 ─ MobilenetV1/Conv2d_1_depthwise/depthwise_weights/read/_79__cf__79
7 MobilenetV1/MobilenetV1/Conv2d_1_depthwise/BatchNorm/batchnorm/mul_1 Mul
└─── 0 ─ MobilenetV1/MobilenetV1/Conv2d_1_depthwise/depthwise
└─── 1 ─ MobilenetV1/MobilenetV1/Conv2d_1_depthwise/BatchNorm/batchnorm/mul/_77

In [3]:
graph = tf.GraphDef()
with tf.gfile.Open('../model/fruits/output_graph_fruits_hidden_1.0_224_q_without_dropout_o.pb', 'r') as f:
    data = f.read()
    graph.ParseFromString(data)
    
display_nodes(graph.node)

0 MobilenetV1/Predictions/Reshape/shape Const
1 MobilenetV1/Conv2d_0/weights/read/_82__cf__82 Const
2 input Placeholder
3 MobilenetV1/MobilenetV1/Conv2d_0/BatchNorm/batchnorm/mul_1 Conv2D
└─── 0 ─ input
└─── 1 ─ MobilenetV1/Conv2d_0/weights/read/_82__cf__82
4 MobilenetV1/MobilenetV1/Conv2d_0/BatchNorm/batchnorm/add_1 Add
└─── 0 ─ MobilenetV1/MobilenetV1/Conv2d_0/BatchNorm/batchnorm/mul_1
└─── 1 ─ MobilenetV1/MobilenetV1/Conv2d_0/BatchNorm/batchnorm/sub/_81__cf__81
5 MobilenetV1/MobilenetV1/Conv2d_0/Relu6 Relu6
└─── 0 ─ MobilenetV1/MobilenetV1/Conv2d_0/BatchNorm/batchnorm/add_1
6 MobilenetV1/MobilenetV1/Conv2d_1_depthwise/depthwise DepthwiseConv2dNative
└─── 0 ─ MobilenetV1/MobilenetV1/Conv2d_0/Relu6
└─── 1 ─ MobilenetV1/Conv2d_1_depthwise/depthwise_weights/read/_79__cf__79
7 MobilenetV1/MobilenetV1/Conv2d_1_depthwise/BatchNorm/batchnorm/mul_1 Mul
└─── 0 ─ MobilenetV1/MobilenetV1/Conv2d_1_depthwise/depthwise
└─── 1 ─ MobilenetV1/MobilenetV1/Conv2d_1_depthwise/BatchNorm/batchnorm/mul/_77

In [7]:
graph = tf.GraphDef()
with tf.gfile.Open('../model/output_graph_veg.pb', 'r') as f:
    data = f.read()
    graph.ParseFromString(data)
    
display_nodes(graph.node)

0 input Placeholder
1 MobilenetV1/Conv2d_0/weights Const
2 MobilenetV1/Conv2d_0/weights/read Identity
└─── 0 ─ MobilenetV1/Conv2d_0/weights
3 MobilenetV1/MobilenetV1/Conv2d_0/convolution Conv2D
└─── 0 ─ input
└─── 1 ─ MobilenetV1/Conv2d_0/weights/read
4 MobilenetV1/Conv2d_0/BatchNorm/beta Const
5 MobilenetV1/Conv2d_0/BatchNorm/beta/read Identity
└─── 0 ─ MobilenetV1/Conv2d_0/BatchNorm/beta
6 MobilenetV1/Conv2d_0/BatchNorm/gamma Const
7 MobilenetV1/Conv2d_0/BatchNorm/gamma/read Identity
└─── 0 ─ MobilenetV1/Conv2d_0/BatchNorm/gamma
8 MobilenetV1/Conv2d_0/BatchNorm/moving_mean Const
9 MobilenetV1/Conv2d_0/BatchNorm/moving_mean/read Identity
└─── 0 ─ MobilenetV1/Conv2d_0/BatchNorm/moving_mean
10 MobilenetV1/Conv2d_0/BatchNorm/moving_variance Const
11 MobilenetV1/Conv2d_0/BatchNorm/moving_variance/read Identity
└─── 0 ─ MobilenetV1/Conv2d_0/BatchNorm/moving_variance
12 MobilenetV1/MobilenetV1/Conv2d_0/BatchNorm/batchnorm/add/y Const
13 MobilenetV1/MobilenetV1/Conv2d_0/BatchNorm/batchnorm/a

└─── 0 ─ MobilenetV1/MobilenetV1/Conv2d_7_pointwise/BatchNorm/batchnorm/add_1
301 MobilenetV1/Conv2d_8_depthwise/depthwise_weights Const
302 MobilenetV1/Conv2d_8_depthwise/depthwise_weights/read Identity
└─── 0 ─ MobilenetV1/Conv2d_8_depthwise/depthwise_weights
303 MobilenetV1/MobilenetV1/Conv2d_8_depthwise/depthwise DepthwiseConv2dNative
└─── 0 ─ MobilenetV1/MobilenetV1/Conv2d_7_pointwise/Relu6
└─── 1 ─ MobilenetV1/Conv2d_8_depthwise/depthwise_weights/read
304 MobilenetV1/Conv2d_8_depthwise/BatchNorm/beta Const
305 MobilenetV1/Conv2d_8_depthwise/BatchNorm/beta/read Identity
└─── 0 ─ MobilenetV1/Conv2d_8_depthwise/BatchNorm/beta
306 MobilenetV1/Conv2d_8_depthwise/BatchNorm/gamma Const
307 MobilenetV1/Conv2d_8_depthwise/BatchNorm/gamma/read Identity
└─── 0 ─ MobilenetV1/Conv2d_8_depthwise/BatchNorm/gamma
308 MobilenetV1/Conv2d_8_depthwise/BatchNorm/moving_mean Const
309 MobilenetV1/Conv2d_8_depthwise/BatchNorm/moving_mean/read Identity
└─── 0 ─ MobilenetV1/Conv2d_8_depthwise/BatchNorm/m

In [8]:
graph = tf.GraphDef()
with tf.gfile.Open('/Users/Yoann/Downloads/model.pb', 'r') as f:
    data = f.read()
    graph.ParseFromString(data)
    
display_nodes(graph.node)

0 input Placeholder
1 MobilenetV1/Conv2d_0/weights Const
2 MobilenetV1/Conv2d_0/weights/read Identity
└─── 0 ─ MobilenetV1/Conv2d_0/weights
3 MobilenetV1/MobilenetV1/Conv2d_0/convolution Conv2D
└─── 0 ─ input
└─── 1 ─ MobilenetV1/Conv2d_0/weights/read
4 MobilenetV1/Conv2d_0/BatchNorm/beta Const
5 MobilenetV1/Conv2d_0/BatchNorm/beta/read Identity
└─── 0 ─ MobilenetV1/Conv2d_0/BatchNorm/beta
6 MobilenetV1/Conv2d_0/BatchNorm/gamma Const
7 MobilenetV1/Conv2d_0/BatchNorm/gamma/read Identity
└─── 0 ─ MobilenetV1/Conv2d_0/BatchNorm/gamma
8 MobilenetV1/Conv2d_0/BatchNorm/moving_mean Const
9 MobilenetV1/Conv2d_0/BatchNorm/moving_mean/read Identity
└─── 0 ─ MobilenetV1/Conv2d_0/BatchNorm/moving_mean
10 MobilenetV1/Conv2d_0/BatchNorm/moving_variance Const
11 MobilenetV1/Conv2d_0/BatchNorm/moving_variance/read Identity
└─── 0 ─ MobilenetV1/Conv2d_0/BatchNorm/moving_variance
12 MobilenetV1/MobilenetV1/Conv2d_0/BatchNorm/batchnorm/add/y Const
13 MobilenetV1/MobilenetV1/Conv2d_0/BatchNorm/batchnorm/a

In [9]:
graph = tf.GraphDef()
with tf.gfile.Open('../model/your_output_model.pb', 'r') as f:
    data = f.read()
    graph.ParseFromString(data)
    
display_nodes(graph.node)

0 input Placeholder
1 MobilenetV1/Conv2d_0/weights_quint8_const Const
2 MobilenetV1/Conv2d_0/weights_min Const
3 MobilenetV1/Conv2d_0/weights_max Const
4 MobilenetV1/Conv2d_0/weights Dequantize
└─── 0 ─ MobilenetV1/Conv2d_0/weights_quint8_const
└─── 1 ─ MobilenetV1/Conv2d_0/weights_min
└─── 2 ─ MobilenetV1/Conv2d_0/weights_max
5 MobilenetV1/Conv2d_0/weights/read Identity
└─── 0 ─ MobilenetV1/Conv2d_0/weights
6 MobilenetV1/MobilenetV1/Conv2d_0/convolution Conv2D
└─── 0 ─ input
└─── 1 ─ MobilenetV1/Conv2d_0/weights/read
7 MobilenetV1/Conv2d_0/BatchNorm/beta_quint8_const Const
8 MobilenetV1/Conv2d_0/BatchNorm/beta_min Const
9 MobilenetV1/Conv2d_0/BatchNorm/beta_max Const
10 MobilenetV1/Conv2d_0/BatchNorm/beta Dequantize
└─── 0 ─ MobilenetV1/Conv2d_0/BatchNorm/beta_quint8_const
└─── 1 ─ MobilenetV1/Conv2d_0/BatchNorm/beta_min
└─── 2 ─ MobilenetV1/Conv2d_0/BatchNorm/beta_max
11 MobilenetV1/Conv2d_0/BatchNorm/beta/read Identity
└─── 0 ─ MobilenetV1/Conv2d_0/BatchNorm/beta
12 MobilenetV1/Conv

└─── 0 ─ MobilenetV1/Conv2d_9_depthwise/BatchNorm/moving_mean_quint8_const
└─── 1 ─ MobilenetV1/Conv2d_9_depthwise/BatchNorm/moving_mean_min
└─── 2 ─ MobilenetV1/Conv2d_9_depthwise/BatchNorm/moving_mean_max
667 MobilenetV1/Conv2d_9_depthwise/BatchNorm/moving_mean/read Identity
└─── 0 ─ MobilenetV1/Conv2d_9_depthwise/BatchNorm/moving_mean
668 MobilenetV1/Conv2d_9_depthwise/BatchNorm/moving_variance_quint8_const Const
669 MobilenetV1/Conv2d_9_depthwise/BatchNorm/moving_variance_min Const
670 MobilenetV1/Conv2d_9_depthwise/BatchNorm/moving_variance_max Const
671 MobilenetV1/Conv2d_9_depthwise/BatchNorm/moving_variance Dequantize
└─── 0 ─ MobilenetV1/Conv2d_9_depthwise/BatchNorm/moving_variance_quint8_const
└─── 1 ─ MobilenetV1/Conv2d_9_depthwise/BatchNorm/moving_variance_min
└─── 2 ─ MobilenetV1/Conv2d_9_depthwise/BatchNorm/moving_variance_max
672 MobilenetV1/Conv2d_9_depthwise/BatchNorm/moving_variance/read Identity
└─── 0 ─ MobilenetV1/Conv2d_9_depthwise/BatchNorm/moving_variance
673 Mo